# Initialization

In [1]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# mpbn init
import mpbn

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

This notebook has been executed using the docker image `colomoto/colomoto-docker:2022-05-01`

<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        primes = None
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start:.2f}\n===")
        except TimeOutException:
            print(f"DNF{'(bnet2primes)' if primes is None else ''}\n===")
        signal.alarm(0)

In [3]:
# compute trapspaces using PyBoolNet directly
def bench_mpbn(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            mbn = mpbn.load(model + ".bnet")
            tspaces = list(mbn.attractors(limit=max))
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start:.2f}\n===")
        except TimeOutException:
            print("DNF\n===")
        signal.alarm(0)

In [4]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120, method="asp"):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            # end = time.perf_counter()
            # print(f"conversion: {end - start}s")
            tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max, method=method)
            end = time.perf_counter()
            print(f"{len(list(tspaces))} trap spaces {end - start:.2f}\n===")
        # we need to catch our TimeOutException, but it raises a Py4J exception too…
        except:
            print("DNF\n===")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [5]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
# models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [6]:
bench_pyboolnet(models)

arellano_rootstem
4 trap spaces 0.08
===
calzone_cellfate
27 trap spaces 0.06
===
dahlhaus_neuroplastoma
32 trap spaces 0.10
===
davidich_yeast
12 trap spaces 0.07
===
dinwoodie_life
7 trap spaces 0.04
===
dinwoodie_stomatal
1 trap spaces 0.04
===
faure_cellcycle
2 trap spaces 0.08
===
grieco_mapk
18 trap spaces 0.10
===
irons_yeast
1 trap spaces 0.08
===
jaoude_thdiff
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 3.17
===
klamt_tcr
8 trap spaces 0.07
===
krumsiek_myeloid
6 trap spaces 0.04
===
multivalued
4 trap spaces 0.04
===
n12c5
5 trap spaces 64.03
===
n3s1c1a
2 trap spaces 0.04
===
n3s1c1b
2 trap spaces 0.03
===
n5s3
3 trap spaces 0.05
===
n6s1c2
3 trap spaces 0.06
===
n7s3
3 trap spaces 0.03
===
raf
2 trap spaces 0.06
===
randomnet_n15k3
3 trap spaces 0.05
===
randomnet_n7k3
10 trap spaces 0.06
===
remy_tumorigenesis
25 trap spaces 3.33
===
saadatpour_guardcell
1 trap spaces 0.05
===
selvaggio_emt
INFO there are possib

In [7]:
bench_mpbn(models)

arellano_rootstem


TypeError: object of type 'generator' has no len()

In [ ]:
bench_trappist(models)

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [7]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [8]:
bench_pyboolnet(models)

/notebook/models/CASCADE3
1 trap spaces 92.75
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/Executable_file_for_Alzheimer_model_BCC
Timeout
DNF
===
/notebook/models/Executable_file_for_cholocystokinin_model_BCC
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.22
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
===
/notebook/models/korkut_2015a
Timeout
DNF
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 0.59
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
DNF
===
/notebook/models/HIV-1
Timeout
DNF
===
/notebook/models/Executable_file_for_CaSQ_derived_MAPK_model_BCC
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 68.62
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
INFO there are possibly 

In [8]:
bench_mpbn(models)

/notebook/models/CASCADE3
1 trap spaces 92.75
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/Executable_file_for_Alzheimer_model_BCC
Timeout
DNF
===
/notebook/models/Executable_file_for_cholocystokinin_model_BCC
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.22
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
===
/notebook/models/korkut_2015a
Timeout
DNF
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 0.59
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
DNF
===
/notebook/models/HIV-1
Timeout
DNF
===
/notebook/models/Executable_file_for_CaSQ_derived_MAPK_model_BCC
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 68.62
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
INFO there are possibly 

In [9]:
bench_trappist(models)

/notebook/models/CASCADE3
1 trap spaces 0.65
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/Executable_file_for_Alzheimer_model_BCC
1000 trap spaces 1.03
===
/notebook/models/Executable_file_for_cholocystokinin_model_BCC
1000 trap spaces 0.73
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
1000 trap spaces 1.07
===
/notebook/models/korkut_2015a
1000 trap spaces 0.61
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.13
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.38
===
/notebook/models/HIV-1
1000 trap spaces 0.87
===
/notebook/models/Executable_file_for_CaSQ_derived_MAPK_model_BCC
1000 trap spaces 0.64
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.15
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.08
===
/notebook/models/INTERFERON-1
1000 trap spaces 0.19
===
/notebook/models/TLGLSurvival
318 trap spaces 0.14
===
/notebook/models/Kynurenine_pathw

In [10]:
bench_trappist(models, max=0)

/notebook/models/CASCADE3
1 trap spaces 0.83
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/Executable_file_for_Alzheimer_model_BCC
Timeout
DNF
===
/notebook/models/Executable_file_for_cholocystokinin_model_BCC
Timeout
DNF
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
===
/notebook/models/korkut_2015a
18556 trap spaces 5.01
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
8192 trap spaces 0.64
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
DNF
===
/notebook/models/HIV-1
39424 trap spaces 7.72
===
/notebook/models/Executable_file_for_CaSQ_derived_MAPK_model_BCC
Timeout
DNF
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
Timeout
DNF
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.06
===
/notebook/models/INTERFERON-1
Timeout
DNF
===
/notebook/models/TLGLSurvival
318 trap spaces 0.14
===
/notebook/models/Kynurenine_pathway_stable
Timeout
DNF
===
/notebook/models/cho_2016_all_pos
13312 tr